In [1]:
import subprocess
import os
from types import SimpleNamespace
import torch
# 상위 폴더로 이동
os.chdir('..')
# GPU device 번호 순서 정리
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
# 첫번째 GPU만 사용
os.environ["CUDA_VISIBLE_DEVICES"]= "0"

## GPU가 할당되었는지 확인하기
GPU를 설정한 이후에 다음과 같은 코드로 device에 cuda (GPU)를 설정하고 device와 current_device()를 출력해봄으로 GPU가 잘 할당되었는지 확인한다.

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

### parameter 설정
딥러닝 모델 훈련에 필요한 설정값을 정의하는 방법에 대해 설명하겠습니다. 이 설정은 모델을 훈련할 때 중요한 역할을 하며, 성능을 최적화하는 데 도움이 됩니다. 

1. model 리스트 설정(models)

    우리는 사전학습된 모델을 이용해서 성능을 높일 것입니다. 각각의 모델은 고유한 구조와 특성을 가지고, 이 모델들은 Huggingface에서 확인하여 사용할 수 있습니다.
    또한 여러가지 모델을 사용해봄으로써 어떤 모델들이 성능이 좋게 나올지 확인해볼 것 입니다.


2. 시드값 설정(seeds)

    시드값은 모델 훈련의 재현성을 보장하기 위해 설정됩니다. 시드를 고정하면 매번 동일한 조건에서 실험이 진행되므로, 결과 비교가 용이해집니다. 또한 seed값마다 다른 성능이 나오고 마지막으로 이들을 앙상블하여 성능을 높이는 것 까지 확인해볼 예정입니다.

3. 배치사이즈 설정(batchsize)

    배치 사이즈는 한 번의 훈련 단계에서 처리할 데이터 샘플의 수를 의미합니다. 배치 사이즈가 크면 훈련 속도가 빨라지지만, 메모리 사용량(VRAM)이 증가할 수 있습니다. 또한 일반적으로 배치 사이즈는 너무 크면 성능이 감소함이 알려져있습니다. 또한 배치사이즈가 작을수록(<128) 학습률에 영향을 많이 받습니다. 

4. 학습률 설정(lr)

    학습률(Learning Rate)은 모델이 얼마나 빠르게 또는 천천히 학습할지를 결정하는 중요한 하이퍼파라미터입니다. 학습률이 너무 크면 훈련이 불안정해지고, 너무 작으면 훈련 속도가 매우 느려질 수 있습니다. 또한 일반적으로 배치 사이즈와 학습률은 양의 상관관계에 있습니다. 

    

5. weight decay 설정

    Weight decay는 기계 학습, 특히 딥러닝에서 모델의 과적합(overfitting)을 방지하기 위해 자주 사용되는 정규화 기법입니다. 이는 모델의 학습 과정에서 가중치(weight)가 지나치게 커지는 것을 억제하여, 모델이 학습 데이터에 지나치게 의존하지 않도록 도와줍니다.

    Weight decay는 기본적으로 비용 함수(cost function)에 가중치의 크기를 제어하는 항(term)을 추가하여 구현됩니다. 이 항은 L2 정규화로도 알려져 있으며, 주로 가중치의 제곱합(squared sum)을 비용 함수에 추가하는 방식입니다. 즉, 모델의 비용 함수는 다음과 같이 변형됩니다:

    
    $$L(w) = L_0(w) + \lambda \sum w_i^2$$
    

    여기서,
    - $L(w)$ 는 정규화된 비용 함수입니다.
    - $L_0(w)$는 원래의 비용 함수(예: 교차 엔트로피, MSE 등)입니다.
    - $w_i$는 모델의 가중치입니다.
    - $\lambda$는 weight decay의 강도를 조절하는 하이퍼파라미터입니다. 이 값이 크면 가중치에 더 큰 페널티를 주고, 작으면 페널티가 줄어듭니다.


6. 추천 파라미터
    bs, lr, wd = (32, 0.03, 1e-5)


- convnext small fp16 기준 17698MB VRAM 필요


In [10]:
# config parameter 설정
config = {
    "models": ["facebook/convnext-small-224"],
    "seeds": [11, 22, 33],
    "batchsize": 32,
    "epochs": 15,
    "early_stopping_epoch": 3,
    "dataset_dir": "data/hfdataset/train_valid",
    "project_name" : "cifar100"
}


### simplenamespace
Python에서 딕셔너리는 키-값 쌍을 저장하는 데 자주 사용됩니다. 하지만 설정 값이나 객체를 다룰 때, 클래스 속성처럼 점 표기법(예: `config.models`)을 사용해 값을 접근하는 것이 더 편리할 수 있습니다. Python의 `types.SimpleNamespace`는 이러한 기능을 제공하여 딕셔너리의 키를 객체 속성처럼 사용할 수 있게 해줍니다.

In [11]:
# SimpleNamespace로 변환
config = SimpleNamespace(**config)

## py 파일과 .ipynb 파일의 비교
train.py와 같은 .py 파일과 Jupyter Notebook의 .ipynb 파일은 둘 다 Python 코드를 작성하고 실행할 수 있지만, 목적과 사용 방식이 다릅니다.

1) 주요 목적
- py 파일: Python 스크립트를 작성하는 가장 일반적인 파일 형식입니다. 주로 독립적으로 실행할 수 있는 코드 작성에 사용되며, 대규모 프로젝트에서 많이 사용됩니다. 코드가 직관적이고 파일 단위로 구성되어 있어서 유지보수가 쉽고, 배포 시에도 매우 적합합니다. 학습 및 훈련 자동화 작업에 많이 사용됩니다.

- ipynb 파일: Jupyter Notebook 파일로, 대화형 코드 실행을 위해 설계되었습니다. 한 셀(Cell) 단위로 코드를 작성하고 바로 실행할 수 있어 데이터 분석, 실험, 시각화 등의 작업에 적합합니다. 주로 학습 과정에서 즉각적인 피드백을 받을 수 있는 환경을 제공합니다.

2) 사용 용도
- py 파일:

    자동화: 반복적으로 실행해야 하는 훈련이나 배포 작업에서 .py 파일이 유용합니다. 학습 파이프라인을 자동화하거나, 여러 모델과 하이퍼파라미터를 반복적으로 실험할 때 주로 사용됩니다.

    대규모 프로젝트: 협업이 필요한 대규모 프로젝트에서는 .py 파일을 사용하는 것이 훨씬 효율적입니다. 모듈화된 코드 관리와 배포가 쉽습니다.
- ipynb 파일:

    연구 및 실험: 중간 결과를 즉시 확인할 수 있고, 데이터를 시각화하면서 실험할 때 유리합니다. 데이터 전처리, 탐색적 데이터 분석(EDA), 모델의 성능 확인을 세밀하게 조정할 수 있습니다.

    강의 및 데모: 시각적이고 설명을 곁들인 코드 셀을 만들 수 있어서, 강의나 실습에 매우 적합합니다.

3) 재사용성과 유지보수
- py 파일: .py 파일은 독립적으로 실행되기 때문에 배포 후에도 쉽게 수정 및 재사용이 가능합니다. 특히 대규모 코드 베이스에서는 각 기능을 모듈화하여 여러 곳에서 재사용할 수 있습니다.

- ipynb 파일: .ipynb 파일은 실험적이고 탐색적인 작업에 더 적합하며, 특정 실험 결과에 집중할 때는 편리합니다. 하지만 대규모 프로젝트에서는 파일 관리와 코드 유지보수가 어렵고, 자동화된 작업이나 재사용성에서는 제한적입니다.



우리는 오늘 .py 파일을 이용해서 실험을 돌려볼것입니다.

In [ ]:
# Loop over models and seeds
for model in config.models:
    project_model_name = model.replace("/", "_")
    for seed in config.seeds:
        command = [
            "CUDA_VISIBLE_DEVICES=0,1",
            "python3", "train.py",
            "-b", str(config.batchsize),
            "-e", str(config.epochs),
            "--use-v2",
            "--seed", str(seed),
            "--model", model,
            "--project-name", f"{config.project_name}_{project_model_name}",
            "--dataset-dir", config.dataset_dir,
            "--fp16",
            "--torch-compile",
        ]
        
        # Use subprocess to execute the command
        subprocess.run(" ".join(command), shell=True, check=True)